In [63]:
!pip install redis

In [64]:
import redis
import json

In [65]:
r = redis.Redis(
    host='redis',
    port=6379,
    db=0,
    decode_responses=True
)

In [77]:
actual_id = 1

def next_id():
    global actual_id
    id = str(actual_id)
    actual_id = actual_id + 1
    return id

In [78]:
def add_player(id, name, country, score):
    player = {
        "name": name,
        "country": country,
        "games_played": 0,
        "score": score
    }
    r.hset(f"player<{id}>", mapping=player)
    r.zadd("leaderboard", {id:score})

In [82]:
add_player(next_id(), "Juan", "España", "0")

In [92]:
def update_score(id, points):

    if not r.exists(f"player<{id}>"):
        print(f"Error: Jugador con ID {id} no encontrado.")
        return

    games = r.hget(f"player<{id}>", "games_played")
    games = int(games) + 1
    r.hset(f"player<{id}>", "games_played", games)

    score = r.hget(f"player<{id}>", "score")
    score = int(score) + points
    r.hset(f"player<{id}>", "score", score)

    r.zadd("leaderboard",{id:points})

In [94]:
update_score(1, 150)

In [106]:
def player_info(id):

    player = f"player<{id}>"

    jugador = r.hgetall(player)
    for key, value in jugador.items():
        print(f" > {key}: {value}")
    

In [107]:
player_info(4)

 > name: Juan
 > country: España
 > games_played: 0
 > score: 0


In [ ]:
def show_top_players(n):
    scores = r.zrevrange("leaderboard", 0, n -1, withscores=True)

    for i, (id, score) in enumerate(scores):

        jugador = r.hgetall(f"player<{id}>")
      
        print(f" #{i+1}: { jugador['name']} ({jugador['country']})")
        print(f"     ID: {id} | Puntuación: {int(jugador['score'])}")
        print("  " + "-"*30)

In [111]:
show_top_players(3)

 #1: Hugo (España)
     ID: 1 | Puntuación: 150
  ------------------------------
 #2: Alvaro (España)
     ID: 3 | Puntuación: 100
  ------------------------------
 #3: Juan (España)
     ID: 4 | Puntuación: 0
  ------------------------------


In [ ]:
def register_login(id):
    id = f"player<{id}>"
    